In [1]:
import glob
import pandas as pd
from collections import Counter

In [2]:
#Read artists master data that will be joined to concatenated list of all social data
artists = pd.read_csv('master_artists_list.csv')

In [3]:
artists.columns

Index(['artist_name', 'age', 'years_active', 'genre_pop', 'genre_rock',
       'genre_hip_hop', 'genre_world_music', 'genre_r_and_b',
       'genre_spiritual', 'genre_electronic', 'genre_jazz', 'genre_country',
       'genre_blues', 'genre_folk', 'facebook', 'instagram', 'twitter',
       'youtube'],
      dtype='object')

In [4]:
artists['artist_name']

0                    Shakira
1                    Beyoncé
2             Jennifer Lopez
3                Miley Cyrus
4          Justin Timberlake
5                      Adele
6                Chris Brown
7             Britney Spears
8           Enrique Iglesias
9                    Pitbull
10                    Maluma
11                 Nicky Jam
12                      Pink
13             Avril Lavigne
14             Calvin Harris
15              Romeo Santos
16                      Zayn
17               Alicia Keys
18                Snoop Dogg
19              Harry Styles
20                     Usher
21              Ricky Martin
22              Mariah Carey
23              Prince Royce
24        Christina Aguilera
25                       Sia
26             Martin Garrix
27                Little Mix
28           Camilla Cabello
29                    Thalia
               ...          
100             Diego Torres
101               Kane Brown
102                   Hozier
103           

In [5]:
#Read all social data files in
files = glob.glob('SocialData/*.csv')
files

['SocialData/twitter_features.csv',
 'SocialData/youtube_to_combine.csv',
 'SocialData/facebook_7yrs.csv',
 'SocialData/instagram_7yrs_cleaned.csv']

In [6]:
df_list = [pd.read_csv(file) for file in files]

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [7]:
sum = 0
for d in df_list:
    print("num columns", len(d.columns))
    sum+=d.shape[0]
    print(d.shape)
    print(d.columns)

print("number of rows in 4 dataframes: ", sum)

num columns 22
(305308, 22)
Index(['Unnamed: 0', 'Twitter_Handle', 'count_of_followers', 'created',
       'description', 'crowdtangle_score', 'description_length',
       'hashtag_count', 'type_link', 'type_photo', 'type_text', 'type_video',
       'created_12am-6am', 'created_12pm-6pm', 'created_6am-12pm',
       'created_6pm-12am', 'time_since_last_post', 'within_week_release',
       'within_month_release', 'social_engagement_score', 'posts_last_7days',
       'posts_last_30days'],
      dtype='object')
num columns 22
(20441, 22)
Index(['artist_name', 'description', 'created', 'social_engagement_score',
       'time_since_last_post', 'time_of_day', 'within_week_release',
       'within_month_release', 'created_12am-6am', 'created_12pm-6pm',
       'created_6am-12pm', 'created_6pm-12am', 'posts_last_7days',
       'posts_last_30days', 'crowdtangle_score', 'type_photo', 'type_video',
       'type_text', 'type_link', 'count_of_followers', 'hashtag_count',
       'description_length'],

In [8]:
#Temporary fixes for youtube
df_list[1].drop(columns='time_of_day', inplace=True)
df_list[1].reset_index(inplace=True)
df_list[1].rename(columns = {'index': 'Unnamed: 0'}, inplace=True)

In [9]:
#Renaming columns that were not aligned
#df_list[0].rename(columns = {'fb_handle_artist_name': 'artist_name'}, inplace=True) #Facebook
df_list[0].rename(columns = {'Twitter_Handle': 'artist_name'}, inplace=True) #Twitter

In [10]:
#Adding Dummy columns to indicate which dataset it is
df_list[0]['Facebook'] = 0
df_list[0]['Instagram'] = 0
df_list[0]['Twitter'] = 1
df_list[0]['Youtube'] = 0

df_list[1]['Facebook'] = 0
df_list[1]['Instagram'] = 0
df_list[1]['Twitter'] = 0
df_list[1]['Youtube'] = 1

df_list[2]['Facebook'] = 1
df_list[2]['Instagram'] = 0
df_list[2]['Twitter'] = 0
df_list[2]['Youtube'] = 0

df_list[3]['Facebook'] = 0
df_list[3]['Instagram'] = 1
df_list[3]['Twitter'] = 0
df_list[3]['Youtube'] = 0

In [11]:
fb = df_list[2].merge(artists, left_on='artist_name', right_on='facebook')
instagram = df_list[3].merge(artists, left_on='artist_name', right_on='instagram')
twitter = df_list[0].merge(artists, left_on='artist_name', right_on='twitter')
yt = df_list[1].merge(artists, left_on='artist_name', right_on='youtube')
socials = [fb, instagram, twitter, yt]

In [12]:
for s in socials:
    print(s.shape)

(297603, 44)
(385381, 44)
(305308, 44)
(20441, 44)


###### Investigation Start

In [31]:
fb_check = df_list[2].merge(artists, how='outer', left_on='artist_name', right_on='facebook')


In [32]:
fb_check.shape

(297603, 44)

In [34]:
#Delta between full outer join and inner join
fb_check.shape[0] - fb.shape[0]

0

In [35]:
fb_diff = pd.concat([fb, fb_check]).drop_duplicates(keep=False)

In [36]:
fb_diff.head()

,Unnamed: 0,artist_name_x,created,description,crowdtangle_score,count_of_followers,type_link,type_photo,type_text,type_video,...,genre_spiritual,genre_electronic,genre_jazz,genre_country,genre_blues,genre_folk,facebook,instagram,twitter,youtube


In [37]:
#Differences match
fb_diff.shape

(0, 44)

In [38]:
fb_diff2 = fb_diff[['artist_name_x', 'facebook', 'instagram', 'twitter', 'youtube']]

In [39]:
#NOTE: THIS is what was missing originally.  Will be blank if re-run
fb_diff2.groupby('artist_name_x').min()

,facebook,instagram,twitter,youtube
artist_name_x,,,,


##### Instagram Check

In [41]:
in_check = df_list[3].merge(artists, how='outer', left_on='artist_name', right_on='instagram')
print(in_check.shape)
#Delta between full outer join and inner join
print(in_check.shape[0] - instagram.shape[0])

in_diff = pd.concat([instagram, in_check]).drop_duplicates(keep=False)

(385381, 44)
0


In [42]:
in_diff.head()

,Unnamed: 0,artist_name_x,count_of_followers,created,description,crowdtangle_score,description_length,hashtag_count,social_engagement_score,type_photo,...,genre_spiritual,genre_electronic,genre_jazz,genre_country,genre_blues,genre_folk,facebook,instagram,twitter,youtube


In [43]:
#Differences match
in_diff.shape

(0, 44)

In [44]:
in_diff2 = in_diff[['artist_name_x', 'facebook', 'instagram', 'twitter', 'youtube']]
in_diff2.groupby('artist_name_x').min()

,facebook,instagram,twitter,youtube
artist_name_x,,,,


###### investigation stop

In [45]:
df = pd.concat(socials)
df.drop('Unnamed: 0',axis=1,inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [46]:
df.sample(10)

,Facebook,Instagram,Twitter,Youtube,age,artist_name_x,artist_name_y,count_of_followers,created,created_12am-6am,...,time_since_last_post,twitter,type_link,type_photo,type_text,type_video,within_month_release,within_week_release,years_active,youtube
182272,0,1,0,0,36,kanygarcia,Kany Garcia,284087.0,2016-10-15 13:28:27,0,...,0 days 00:02:28.000000000,kanygarcia,0,1,0,0,0,0,13,Kany Garcia
118610,1,0,0,0,39,SilvestreDangond,Silvestre Dangond,1895922.0,2/15/18 0:33,1,...,2 days 16:03:41.000000000,SilvestreFDC,0,1,0,0,0,0,18,Silvestre Dangond
32658,1,0,0,0,35,FutureOfficial,Future,6995329.0,1/19/17 23:14,0,...,18 days 18:13:36.000000000,1future,0,1,0,0,0,0,10,Future
365711,0,1,0,0,40,usher,Usher,3145057.0,2015-10-03 04:34:05,1,...,0 days 00:14:25.000000000,Usher,0,0,0,1,0,0,27,Usher
283368,1,0,0,0,50,victormanuelleonline,Victor Manuelle,1148020.0,12/27/15 23:24,0,...,1 days 01:51:09.000000000,VictorManuelle,0,0,0,1,0,0,34,Victor Manuelle
6214,1,0,0,0,71,AliceCooper,Alice Cooper,4410585.0,12/11/16 22:00,0,...,0 days 01:00:01.000000000,alicecooper,0,1,0,0,0,0,57,Official Alice Cooper
195354,0,1,0,0,23,luiscoronelmusic,Luis Coronel,2102594.0,2017-07-09 02:00:55,1,...,0 days 05:09:53.000000000,LuisCoronel,0,1,0,0,0,0,7,luis coronel
98977,1,0,0,0,78,RobertoCarlosOficial,Roberto Carlos,5089394.0,3/30/15 23:38,0,...,0 days 03:08:01.000000000,robertocarlos,1,0,0,0,0,0,68,Roberto Carlos
70190,0,0,1,0,26,LittleMix,Little Mix,8157861.0,2016-11-16 20:45:06,0,...,0 days 01:23:06.000000000,LittleMix,0,1,0,0,0,0,8,Little Mix
16025,0,0,0,1,42,Shakira Mebarak,Shakira,0.0,5/12/2017 16:53,0,...,0 days 00:00:12.000000000,shakira,0,0,0,0,0,0,31,Shakira Mebarak


In [47]:
print(df.shape)
print("number of rows lost: ", sum - df.shape[0], ", ", (sum - df.shape[0])/sum)

(1008733, 43)
number of rows lost:  0 ,  0.0


In [48]:
df.head(3).T

,0,1,2
Facebook,1,1,1
Instagram,0,0,0
Twitter,0,0,0
Youtube,0,0,0
age,26,26,26
artist_name_x,21Savage,21Savage,21Savage
artist_name_y,21 Savage,21 Savage,21 Savage
count_of_followers,NaN,NaN,NaN
created,5/18/15 19:24,5/25/15 16:00,5/25/15 22:26
created_12am-6am,0,0,0


In [196]:
# df.to_csv('Combined_Social_Data.csv')

In [49]:
df.to_pickle("Combined_Social_Data_Pickle.pkl")

In [51]:
df.shape

(1008733, 43)